#### Simple Gen AI APP Using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GEMINI_API_KEY']=os.getenv("GEMINI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [25]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/Supervised_learning")
loader

In [26]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Supervised_learning', 'title': 'Supervised learning - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nSupervised learning - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\

In [27]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [28]:
documents

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Supervised_learning', 'title': 'Supervised learning - Wikipedia', 'language': 'en'}, page_content='Supervised learning - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(To

In [29]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",   # Gemini embedding model
    google_api_key=os.getenv("GEMINI_API_KEY")
)

In [30]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [31]:
vectorstoredb

In [ ]:
## Query From a vector db
query="What is presence of non-linearity in supervised learning?"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'Presence of interactions and non-linearities. If each of the features makes an independent contribution to the output, then algorithms based on linear functions (e.g., linear regression, logistic regression, support-vector machines, naive Bayes) and distance functions (e.g., nearest neighbor methods, support-vector machines with Gaussian kernels) generally perform well. However, if there are complex interactions among features, then algorithms such as decision trees and neural networks work better, because they are specifically designed to discover these interactions. Linear methods can also be applied, but the engineer must manually specify the interactions when using them.'

In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    google_api_key=os.getenv("GEMINI_API_KEY")  
)

print(llm)


model='models/gemini-1.5-flash' google_api_key=SecretStr('**********') temperature=0.0 client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000023C003A8F90> default_metadata=()


In [35]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000023C003A8F90>, default_metadata=())
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [39]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"What all steps to be followed to solve given problem of supervised learning",
    "context":[Document(page_content=""" To solve a given problem of supervised learning, the following steps must be performed:

Determine the type of training samples. Before doing anything else, the user should decide what kind of data is to be used as a training set. In the case of handwriting analysis, for example, this might be a single handwritten character, an entire handwritten word, an entire sentence of handwriting, or a full paragraph of handwriting.
Gather a training set. The training set needs to be representative of the real-world use of the function. Thus, a set of input objects is gathered together with corresponding outputs, either from human experts or from measurements.
Determine the input feature representation of the learned function. The accuracy of the learned function depends strongly on how the input object is represented. Typically, the input object is transformed into a feature vector, which contains a number of features that are descriptive of the object. The number of features should not be too large, because of the curse of dimensionality; but should contain enough information to accurately predict the output.
Determine the structure of the learned function and corresponding learning algorithm. For example, one may choose to use support-vector machines or decision trees.
Complete the design. Run the learning algorithm on the gathered training set. Some supervised learning algorithms require the user to determine certain control parameters. These parameters may be adjusted by optimizing performance on a subset (called a validation set) of the training set, or via cross-validation.
Evaluate the accuracy of the learned function. After parameter adjustment and learning, the performance of the resulting function should be measured on a test set that is separate from the training set. """
)]
})

'Based on the provided text, to solve a supervised learning problem, one must:\n\n1.  Determine the type of training samples.\n2.  Gather a training set.\n3.  Determine the input feature representation of the learned function.\n4.  Determine the structure of the learned function and corresponding learning algorithm.\n5.  Complete the design (run the learning algorithm and adjust parameters using a validation set or cross-validation).\n6.  Evaluate the accuracy of the learned function on a separate test set.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [46]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [ ]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)## here document chain gives the context information


In [48]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023C00373810>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
  

In [49]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"What all steps to be followed to solve given problem of supervised learning"})
response['answer']

'Based on the provided text, the steps to solve a supervised learning problem are:\n\n1. **Determine the input feature representation:** Transform the input object into a feature vector with an appropriate number of descriptive features.\n2. **Determine the structure of the learned function and algorithm:** Choose an appropriate algorithm (e.g., support-vector machines or decision trees).\n3. **Complete the design and run the algorithm:** Run the chosen algorithm on the training set, adjusting control parameters as needed using a validation set or cross-validation.\n4. **Evaluate accuracy:** Measure the performance of the learned function on a separate test set.'

In [50]:

response

{'input': 'What all steps to be followed to solve given problem of supervised learning',
 'context': [Document(id='8e8ce1fb-dd4e-4630-8dc9-25741d5b1051', metadata={'source': 'https://en.wikipedia.org/wiki/Supervised_learning', 'title': 'Supervised learning - Wikipedia', 'language': 'en'}, page_content='Steps to follow[edit]\nTo solve a given problem of supervised learning, the following steps must be performed:'),
  Document(id='4014043b-e4e6-4713-bc74-57e5befe15ae', metadata={'source': 'https://en.wikipedia.org/wiki/Supervised_learning', 'title': 'Supervised learning - Wikipedia', 'language': 'en'}, page_content='Evaluate the accuracy of the learned function. After parameter adjustment and learning, the performance of the resulting function should be measured on a test set that is separate from the training set.\nAlgorithm choice[edit]\nA wide range of supervised learning algorithms are available, each with its strengths and weaknesses. There is no single learning algorithm that works

In [51]:
response['context']

[Document(id='8e8ce1fb-dd4e-4630-8dc9-25741d5b1051', metadata={'source': 'https://en.wikipedia.org/wiki/Supervised_learning', 'title': 'Supervised learning - Wikipedia', 'language': 'en'}, page_content='Steps to follow[edit]\nTo solve a given problem of supervised learning, the following steps must be performed:'),
 Document(id='4014043b-e4e6-4713-bc74-57e5befe15ae', metadata={'source': 'https://en.wikipedia.org/wiki/Supervised_learning', 'title': 'Supervised learning - Wikipedia', 'language': 'en'}, page_content='Evaluate the accuracy of the learned function. After parameter adjustment and learning, the performance of the resulting function should be measured on a test set that is separate from the training set.\nAlgorithm choice[edit]\nA wide range of supervised learning algorithms are available, each with its strengths and weaknesses. There is no single learning algorithm that works best on all supervised learning problems (see the No free lunch theorem).\nThere are four major issue